In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn import impute
from sklearn.pipeline import Pipeline
from sklearn import compose
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import median_absolute_error
# from sklearn.model_selection import cross_val_score

In [2]:
from sklearn.metrics import median_absolute_error

In [3]:
data = pd.read_csv("../data/data_cleaned.csv")

In [4]:
df = data

In [5]:
# replace nan
cat_col = ['Nationality', 'Club', 'Preferred Foot', 'Off Work Rate','Def Work Rate', 'Body Type', 'Real Face', 'Position', 'Loaned Out']
df[cat_col] = df[cat_col].fillna(value='nan')  # fill na for categorical features
# df = df.fillna(value=df.median())  # fill na for numerical features

In [6]:
# label encoding
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for c in cat_col:
    df[c] = le.fit_transform(df[c])

In [7]:
regressor = RandomForestRegressor(n_estimators=30,
                                  min_samples_leaf=1,
                                  random_state=42)

In [8]:
numerical_col =  list(set(df.columns) - set(cat_col))
numerical_col.remove('Value')

In [9]:
def make_pipeline(regressor=None):
    "Create a single pipeline that processing the data and then fits the regressor." 
    
    median_impute = impute.SimpleImputer(strategy='median')

    numerical_names = numerical_col
    
    col_transform = compose.ColumnTransformer([('numerics',median_impute,numerical_names)])

    steps_list = [('col_t',col_transform),('clf',regressor)]

    pipeline = Pipeline(steps=steps_list)
    
    return pipeline

In [10]:
pipeline = make_pipeline(regressor)

In [11]:
y = df['Value']

In [12]:
X = df.drop('Value',axis=1)

In [13]:
kfold= StratifiedKFold(n_splits=5,shuffle=True)

In [14]:
scores = cross_val_score(pipeline, X, y, cv=kfold,scoring=make_scorer(median_absolute_error))

/home/sean-cx1/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


In [15]:
np.mean(scores)

22999.999999999993

In [16]:
scores

array([22500.        , 23333.33333333, 23333.33333333, 23333.33333333,
       22500.        ])